In [8]:
# use natural language toolkit
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
import openpyxl
import numpy as np
import time

stemmer = LancasterStemmer()

In [13]:
from nltk.corpus import brown

In [14]:
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [12]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [9]:
# 2 classes of training data

wb = openpyxl.load_workbook(r'Trainingsdaten_mit_Abnahmeprotokoll.xlsx')
sheet = wb['Tabelle1']

training_data = []
training_data_value = []
def removeDuplicates(listofElements):    
    # Create an empty list to store unique elements
    uniqueList = []
    # Iterate over the original list and for each element
    # add it to uniqueList, if its not already there.
    for elem in listofElements:
        if elem not in uniqueList:
            uniqueList.append(elem)    
    # Return the list of unique elements        
    return uniqueList

for i in range(250):
    gwm_dauer = sheet.cell(row=i+2, column=15).value
    gwm_text = sheet.cell(row=i+2, column=4).value + ", " + sheet.cell(row=i+2, column=5).value
    training_data_value.append(sheet.cell(row=i+1, column=11).value)
    training_data.append({"class":str(gwm_dauer), "sentence":str(gwm_text)})
    
print ("%s sentences in training data" % len(training_data))
print(training_data)

250 sentences in training data
[{'class': '48', 'sentence': 'Bodenbelags Arbeiten, Umbau Verwaltungsgebäude'}, {'class': '48', 'sentence': 'Duschtrennwände, Ferienhaus Titisee'}, {'class': '24', 'sentence': 'USV - Anlage, Generalüberholung und Wiederinbetriebnahme USV Geb.5'}, {'class': '24', 'sentence': 'Elektrotechnische Anlage, Ertüchtigung der Unterverteiler im Verwaltungsgebäude'}, {'class': '24', 'sentence': 'Elektrotechnische Anlage, Elektroverkabelung für Mietkältemaschine'}, {'class': '48', 'sentence': 'sanitäre Anlagen, Austausch der Trinkwasserleitung in der Garage im 2.Riegel Kern 1 beim Vorstandsbereich'}, {'class': '48', 'sentence': 'Außenanlagen, Sanierung Bürogebäude Kurt-Schumacher-Str.39'}, {'class': '48', 'sentence': 'Beleuchtungsanlage, Sanierung Bürogebäude Kurt-Schumacher-Str.39'}, {'class': '48', 'sentence': 'Böden, Sanierung Bürogebäude Kurt-Schumacher-Str.39'}, {'class': '48', 'sentence': 'Decke, Sanierung Bürogebäude Kurt-Schumacher-Str.39'}, {'class': '48', '

In [15]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our training data
for pattern in training_data:
    # tokenize each word in the sentence
    w = nltk.word_tokenize(pattern['sentence'])
    # add to our words list
    words.extend(w)
    # add to documents in our corpus
    documents.append((w, pattern['class']))
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

# remove duplicates
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

250 documents
2 classes ['48', '24']
599 unique stemmed words ['druckschläuch', 'bzw', 'an', 'hup', 'divers', 'ausbesserungsarbeit', 'inbetriebnahm', 'bruchsalerstraß', 'schachttürflügel', '93', 'wand', 'wärmeabzug', '500', 'polo', 'brandabschnit', 'leonberg', 'dämmung', '2/3', 'rohrrahmentür', 'splitgerät', 'radlerdusch', 'für', 'elektroinstallationsarbeit', 'rr2', 'aufzugsbrems', 'hebeanl', 'for', 'hardw', 'kre', 'brandfrüherkennung', 'liefern', 'fliesenlegearbeit', 'dialog', 'sanitäranl', 'ausgangskontaktschl', '2.1', 'autom', 'ersatzvornahm', 'stuttgart', 'sonnenschutzsystem', 'elektroanl', 'ddc', 'gebäud', 'motorradstellplätz', '122', 'fenst', 'herstellung', 'triebwerksbrems', 'parkplätz', 'bau', 'hausalarmsystem', 'umbau', 'bauwerk', 'lichtenbergerst', '134', 'f0', 'rückbau', 'putzarbeit', 'loung', 'lagergebäud', 'bgv', 'gitterrost', 'klimagerät', 'viessman', 'schiebetoranlage', '3', 'freudenstadt', 'förderanlage', 'brandschutztür', 'meteorit', 'au', 'bau11', 'neubau', 'führungse

In [16]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)

print ("# words", len(words))
print ("# classes", len(classes))

# words 599
# classes 2


In [17]:
# sample training/output
i = 5
w = documents[i][0]
print ([stemmer.stem(word.lower()) for word in w])
print (training[i])
print (output[i])

['sanitär', 'anl', ',', 'austausch', 'der', 'trinkwasserleitung', 'in', 'der', 'gar', 'im', '2.riegel', 'kern', '1', 'beim', 'vorstandsbereich']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    # input layer is our bag of words
    l0 = x
    # matrix multiplication of input and hidden layer
    l1 = sigmoid(np.dot(l0, synapse_0))
    # output layer
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [19]:
# ANN and Gradient Descent code from https://iamtrask.github.io//2015/07/27/python-network-part2/
def train(X, y, hidden_neurons=10, alpha=1, epochs=10000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # how much did we miss the target value?
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            # if this 10k iteration's error is greater than the last iteration, break out
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    # persist synapses
    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)

In [20]:
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=20, alpha=0.1, epochs=100000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 20 neurons, alpha:0.1, dropout:False 
Input matrix: 250x599    Output matrix: 1x2
delta after 10000 iterations:0.005538856476818987
delta after 20000 iterations:0.005052686997040612
delta after 30000 iterations:0.0048445850524167465
delta after 40000 iterations:0.004722880151616922
delta after 50000 iterations:0.004640908655689383
delta after 60000 iterations:0.004580998494338704
delta after 70000 iterations:0.004534805583884937
delta after 80000 iterations:0.004497816092750237
delta after 90000 iterations:0.004467349641409001
delta after 100000 iterations:0.0044417022637353665
saved synapses to: synapses.json
processing time: 148.0025713443756 seconds


In [22]:
wb2 = openpyxl.load_workbook(r'Testdaten_mit_Abnahmeprotokoll.xlsx')
sheet2 = wb2['Tabelle1']
test_data = []
test_data_result = []
test_data_value = []

for i in range(99):
    gwm_text = sheet2.cell(row=i+2, column=4).value + ", " + sheet2.cell(row=i+2, column=5).value
    test_data.append(str(gwm_text))
    
    gwm_dauer =  sheet2.cell(row=i+2, column=15).value 
    test_data_result.append(str(gwm_dauer))
    test_data_value.append(str(sheet2.cell(row=i+1, column=11).value))

print(test_data_value)
# probability threshold
ERROR_THRESHOLD = 0.2
# load our calculated synapse values
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    print ("correct answer: " + test_data_result[i])
    return return_results

def query(sentence, show_details=False):
    results = think(sentence, show_details)
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

scorecard = []

for i in range(len(test_data)):
#    if float(test_data_value[i]) <= 40000.00:
    results = classify(str(test_data[i]))
#    else:
#        results = ["48" , 1.00]
    
    try:
        if results[0][0] == test_data_result[i]:
            scorecard.append("1")
        else: 
            scorecard.append("0")
    except:
        scorecard.append("0")

print(scorecard)

sums = 0
for i in range(len(scorecard)):
    sums += int(scorecard[i])

accuracy = sums / len(scorecard)
print("accuracy: " +  str(accuracy))

print ()
classify(test_data[25], show_details=True)
classify("Sanierung des  Bodens im  Müllraum im  Bereich vor den  Schienen", show_details=True)

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
Personenaufzug, Abdichten des Getriebes Aufzugsmotor 95/1735 
 classification: [['24', 0.8282471645850693]]
correct answer: 24
Personenaufzug, Abdichten des Getriebes Aufzugsmotor 95/1732 
 classification: [['24', 0.7222552534222068], ['48', 0.2542902321117814]]
correct answer: 24
Personenaufzug, Lagerwechsel Aufzugsmotor 95/1741 
 classification: [['24', 0.6382242387355141], ['48', 0.2988446903895437]]
correct answer: 24
Personenaufzug, Lagerwechsel Aufzugsmotor 95/1739 
 classification: [['24', 0.6382242387355141], ['48', 0.2988446903895437]]
correct answer: 24
Personenaufzug, Abdichten des Get

[['48', 0.9593932401103382]]

In [277]:
wb3 = openpyxl.load_workbook(r'C:\Users\Simon\Desktop\GWM_extractor\GWM_Daten.xlsx')
sheet3 = wb3['GWM Daten']
test_data_2 = []
for i in range(5):
    gwm_text = sheet3.cell(row=i+2, column=4).value
    test_data_2.append(str(gwm_text))

for document in test_data_2:
    query(document , show_details=False)


Bestellung laut  Ihrem  Angebot  Nr. 160904 vom 26.09.2017 D U A 87  Gebäude 5+6:  Sanierung  Elektro- Unterverteiler  Mängelbeseitigung D G U V  V3. 
 classification: [['24', 0.5959413788507242], ['48', 0.331339420832572]]
Ausbildung  Heilbronn  Liefern und  Montieren von  Brandschutztüren  
 classification: [['48', 0.8528891387738269]]
Instandsetzung der  Nachteingangstüre im  Foyer 
 classification: [['24', 0.9999979857449355]]
Sanierung des  Bodens im  Müllraum im  Bereich vor den  Schienen 
 classification: [['24', 0.7815482250174517]]
Instandsetzung der  Fenster nach durchgeführter  Wartung im  Bereich Hochhaus 
 classification: [['24', 0.9999744367190314]]
